In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
import sys
sys.path.insert(0, "python")
from network import *
from protocols import *
from adversary import *
from simulator import *
from experiments import run_and_eval, run_experiment

In [ ]:
def shorten_protocol_name(x: str):
    val = x.replace("Protocol","").replace("spreading_proba","p")
    val = val.split("broadcast")[0][:-1].replace("(", ": ")
    if val[-1] == ",":
        val = val[:-1]
    return val

In [ ]:
nw_generator = NodeWeightGenerator("random")
ew_generator = EdgeWeightGenerator("normal")

# Dandelion example

In [ ]:
net = Network(nw_generator, ew_generator, num_nodes=1000, k=20)

In [ ]:
dp = DandelionProtocol(net, 0.5)

dp = DandelionPlusPlusProtocol(net, 0.5)

nx.draw(dp.anonymity_graph, node_size=20)

In [ ]:
%%time
adv = Adversary(net, 0.1)
sim = Simulator(dp, adv, 30, verbose=False)
sim.run()

In [ ]:
%%time
evaluator = Evaluator(sim, "first_reach")
print(evaluator.get_report())

In [ ]:
%%time
evaluator = Evaluator(sim, "first_sent")
print(evaluator.get_report())

In [ ]:
%%time
evaluator = Evaluator(sim, "dummy")
print(evaluator.get_report())

# Spreading time experiment

In [ ]:
%%time
bc_mode="sqrt"
net = Network(nw_generator, ew_generator, num_nodes=100, k=10)
print(net.num_nodes)
protocols = [
    DandelionProtocol(net, 0.5, broadcast_mode=bc_mode),
    DandelionProtocol(net, 0.25, broadcast_mode=bc_mode),
    DandelionPlusPlusProtocol(net, 0.5, broadcast_mode=bc_mode),
    DandelionPlusPlusProtocol(net, 0.25, broadcast_mode=bc_mode),
    BroadcastProtocol(net, broadcast_mode=bc_mode)
]

for protocol in protocols:
    sim = Simulator(protocol, adv, 10, verbose=False)
    msg_cov = sim.run(coverage_threshold=1.0)
    #print(msg_cov)
    print(np.mean(msg_cov))
    x = np.arange(0.1,1.0,0.1)
    mean_quantiles, std_quantiles = sim.node_contact_time_quantiles(x)
    name = shorten_protocol_name(str(protocol)) if not "Broadcast" in str(protocol) else str(protocol)
    plt.plot(x, mean_quantiles, label=name)
plt.legend()
plt.ylabel("time (ms)")
plt.xlabel("quantiles")

# Unittest in the making

In [ ]:
import sys, os, pytest

sys.path.insert(0, "%s/python" % os.getcwd())
import networkx as nx
from network import Network, NodeWeightGenerator, EdgeWeightGenerator
from simulator import Simulator, Evaluator
from message import Message
from protocols import BroadcastProtocol, DandelionProtocol, DandelionPlusPlusProtocol
from adversary import Adversary

In [ ]:
### First sent vs First reach test
rnd_node_weight = NodeWeightGenerator("random")
rnd_edge_weight = EdgeWeightGenerator("random")
G = nx.DiGraph()
G.add_nodes_from([1, 2, 3])
G.add_weighted_edges_from([(1, 2, 0.9), (1, 3, 1.84),(2, 3, 1.01)], weight="latency")
print(nx.draw(G,with_labels=True))
net = Network(rnd_node_weight, EdgeWeightGenerator("custom"), graph=G)
protocol = BroadcastProtocol(net, seed=44)
adv = Adversary(net, ratio=0.0, adversaries=[2])
print(adv.nodes)
assert 3 in adv.nodes

# start a message from Node 1
msg = Message(1)
receiver_order = [1, 2, 2, 2]
for i, receiver in enumerate(receiver_order):
    msg.process(protocol, adv)
    print(i,receiver,msg.history)
    assert receiver in msg.history
    # assert len(msg.history) == i + 1

In [ ]:
sim = Simulator(protocol, adv, 1, True)
sim.run(1.0)
for estimator in ["first_reach", "first_sent"]:
    evaluator = Evaluator(sim, estimator)
    results = [
        evaluator.exact_hits,
        evaluator.ranks,
        evaluator.inverse_ranks,
        evaluator.entropies,
    ]
    for i in results:
        print(estimator, i)